In [17]:
allcols = set()
allcolsls = []

In [18]:
import numpy as np
import pandas as pd
import pylab as plt
from tqdm.auto import tqdm
from astropy.io import ascii

In [19]:
import sys

sys.path.append("../")
import sidhelpers

## 1. See how many spectra (%) can be opened by Astropy

In [20]:
# Read spectra metadata
spectra_info = pd.read_csv("../1. download ALL wise data/wiserep_spectra_combined.csv")

# keep p60 only
spectra_info = spectra_info[spectra_info["Telescope"] == "P60"]

In [21]:
spectra_info["Obj. Type"].value_counts()

Obj. Type
SN Ia                4769
SN II                 782
SN Ic                  92
SN Ia-91T-like         90
SN IIn                 62
SN IIb                 54
SN Ib                  53
SN Ic-BL               52
SN IIP                 51
SN Ia-91bg-like        31
TDE                    30
SN Ia-pec              27
SN                     22
CV                     20
Nova                   18
SLSN-II                15
SN Ib/c                15
SN Ibn                 14
SN I                   14
SLSN-I                 13
AGN                    12
SN Icn                 10
SN Iax[02cx-like]       9
SN Ia-CSM               9
Galaxy                  3
SN II-pec               3
QSO                     3
SN Ib-Ca-rich           2
ILRT                    1
SN IIL                  1
LBV                     1
SN Ic-Ca-rich           1
SN Ic-pec               1
Computed-Ia             1
FBOT                    1
SN Ia-SC                1
Name: count, dtype: int64

In [6]:
filenames = spectra_info["Ascii file"].values

In [7]:
# df = ascii.read(f"../1. download ALL wise data/wiserep_data/spectra/{fn}").to_pandas()

In [8]:
# spectra_info["Spec. units"].value_counts().index[0]

In [9]:
# most efficient way to iterate
# https://stackoverflow.com/questions/7837722/what-is-the-most-efficient-way-to-loop-through-dataframes-with-pandas

flux_counter = 0
readerror_counter = 0
errs = []

for fn, wl_unit, spec_unit, flux_ucoeff, lambda_min, lambda_max, del_lambda in tqdm(
    zip(
        spectra_info["Ascii file"],
        spectra_info["WL Units"],
        spectra_info["Spec. units"],
        spectra_info["Flux Unit Coefficient"],
        spectra_info["Lambda-min"],
        spectra_info["Lambda-max"],
        spectra_info["Del-Lambda"],
    ),
    total=len(spectra_info),
):
    try:
        df = sidhelpers.read_spectra(
            f"../1. download ALL wise data/wiserep_data/spectra/{fn}"
        )
        allcols.update(df.columns)
        allcolsls = allcolsls + list(df.columns)
    except Exception as ee:
        readerror_counter = readerror_counter + 1
        errs.append([fn, ee])
        continue
    try:
        sidhelpers.check_spectrafile(df, wl_unit, spec_unit, lambda_min, lambda_max, del_lambda)
    except Exception as e:
        errs.append([fn, e])

        if e == "Flux is not positive!":
            flux_counter = flux_counter + 1
        continue

  0%|          | 0/54005 [00:00<?, ?it/s]

In [10]:
flux_counter

0

In [11]:
df_unique = pd.DataFrame({"columns": list(allcols)})
df_unique.to_csv("unique_columns_ALL_16april.csv", index=False)
df_unique

,columns
0,flux_density
1,True
2,OBJECT
3,ATLASgqg
4,tpk
...,...
500,vacuum_wave
501,AT2020zlp
502,AT2020tul
503,Wavelength_Angstrom


In [12]:
errdf = pd.DataFrame(errs, columns=["Ascii file", "error"]).set_index("Ascii file")

In [13]:
errdf.to_csv("errdf_16april.csv")

In [14]:
errdf

,error
Ascii file,
2006kv_2006-10-18_05-27-54_ESO-NTT_EMMI_NTT-NOT.ascii,Flux units are Other and don't follow an expec...
NN3518_2015-10-21_07-32-31_VLT-UT2_X-Shooter_None.txt,Last wavelength 24789.6 exceeds maximum allowe...
PTF10qyr_2010-08-17_00-00-00_Lick-3m_KAST_PTF.ascii,Flux units are Other and don't follow an expec...
SN2005ad_2005-03-11_00-00-00_Keck1_LRIS_LOSS.flm,Flux units are Other and don't follow an expec...
1998en_1998-11-25_08-11-02_Lick-3m_KAST_BSNIP.flm,Flux units are Other and don't follow an expec...
...,...
SN2022kae_2022-12-04_00-00-00_MMT_BINOSPEC_TNS.dat,All spectrum readers failed:\nread: \nERROR: U...
SN2024aewt_2024-12-27_13-01-14_UH88_SNIFS_TNS.dat,Flux is not positive!
SN2025zi_2025-01-23_01-02-59_Magellan-Baade_IMACS_TNS.dat,All spectrum readers failed:\nread: \nERROR: U...


In [15]:
# import astropy.units as u
# import astropy.constants as const
# from astropy.units import equivalencies

# # Define the flux in mJy
# flux_mJy = 1.0 * u.mJy  # Example value of 1 mJy

# # To convert between frequency and wavelength units, we need to specify the wavelength
# wavelength = 5500 * u.AA  # Example: optical wavelength at 5500 Angstroms

# # Create the equivalency between frequency and wavelength flux densities
# # This accounts for the non-linear relationship between frequency and wavelength
# fnu_to_flambda = u.spectral_density(wavelength)

# # Do the conversion
# flux_flambda = flux_mJy.to(u.erg / u.s / u.cm**2 / u.AA, equivalencies=fnu_to_flambda)

# print(f"Flux density at {wavelength}:")
# print(f"{flux_mJy} = {flux_flambda}")